# Simulacija redova i mreža redova
### Mihael Miličević, kolegij Stohastičko modeliranje, ak. god. 2023./2024.

In [1]:
import numpy as np
import math
from distribution_functions import exponential
from queues import QueueNetwork

## Kendallova notacija

### $ A/B/s/m $

$ A $ - proces koji generira zahtjeve
$ B $ - proces koji obrađuje zahtjeve
$ s $ - broj poslužitelja
$ m $ - kapacitet spremnika

## $ M/M/\infty $
Sustav bez čekanja i gubitaka.

In [2]:
queue_network = QueueNetwork()

generator_scale = 2
consumer_scale = 1

queue = queue_network.create_queue(
    generator_function=exponential(scale=generator_scale),
    consumer_function=exponential(scale=consumer_scale),
    number_of_processors=np.inf,
    buffer_size=np.inf,
)

queue_network.set_transition(
    queue=queue,
    transition={
        -1: 1,
        0: 0
    }
)

ro = consumer_scale / generator_scale

end_time = 100000
queue_network.simulate(end_time)

pi = [np.power(ro, i) * np.exp(-ro) / math.factorial(i) for i in range(6)]
_pi = [queue_network.logger.calculate_stationary_distribution(0, end_time, i) for i in range(6)]

print(f'Teoretski rezultat o stacionarnoj distribuciji broja poslova u sustavu: \n{pi}')
print(f'Eksperimentalni rezultat o stacionarnoj distribuciji  broja poslova u sustavu: \n{_pi}')
print()

average_number_of_busy_processors = queue_network.logger.calculate_average_number_of_busy_processors(0, end_time)
print(f'Teoretski rezultat o očekivanom broju zauzetih procesora: \n{ro}')
print(f'Eksperimentalni rezultat o očekivanom broju zauzetih procesora \n{average_number_of_busy_processors}')
print()

Teoretski rezultat o stacionarnoj distribuciji broja poslova u sustavu: 
[0.6065306597126334, 0.3032653298563167, 0.07581633246407918, 0.012636055410679864, 0.001579506926334983, 0.0001579506926334983]
Eksperimentalni rezultat o stacionarnoj distribuciji  broja poslova u sustavu: 
[0.6061875075074897, 0.3038266674558404, 0.07596868121181023, 0.012350183790952094, 0.001465282898886025, 0.00015648754652735193]

Teoretski rezultat o očekivanom broju zauzetih procesora: 
0.5
Eksperimentalni rezultat o očekivanom broju zauzetih procesora 
0.4996304312104788


## $ M/M/s/0 $
Sustav s gubitcima

In [3]:
queue_network = QueueNetwork()

generator_scale = 2
consumer_scale = 1
s = 3

queue = queue_network.create_queue(
    generator_function=exponential(scale=generator_scale),
    consumer_function=exponential(scale=consumer_scale),
    number_of_processors=s,
    buffer_size=0
)

queue_network.set_transition(
    queue=queue,
    transition={
        -1: 1,
        0: 0
    }
)

end_time = 100000
queue_network.simulate(end_time)

pi_0 = [1 / sum([np.power(ro, i) / math.factorial(i) for i in range(s + 1)])]
pi_i = [pi_0[0] * np.power(ro, i) / math.factorial(i) for i in range(1, s + 1)]
pi = pi_0 + pi_i
_pi = [queue_network.logger.calculate_stationary_distribution(0, end_time, i) for i in range(s + 1)]

print(f'Teoretski rezultat o stacionarnoj distribuciji  broja poslova u sustavu: \n{pi}')
print(f'Eksperimentalni rezultat o stacionarnoj distribuciji  broja poslova u sustavu: \n{_pi}')
print()

average_number_of_busy_processors = queue_network.logger.calculate_average_number_of_busy_processors(0, end_time)
print(f'Teoretski rezultat o očekivanom broju zauzetih procesora: \n{ro * (1 - pi[-1])}')
print(f'Eksperimentalni rezultat o očekivanom broju zauzetih procesora: \n{average_number_of_busy_processors}')
print()

probability_of_loosing_a_job = queue_network.logger.calculate_probability_of_loosing_a_job(0)
print(f'Teoretski rezultat o vjerojatnosti gubitka posla: \n{pi[-1]}')
print(f'Eksperimentalni rezultat o vjerojatnosti gubitka posla: \n{probability_of_loosing_a_job}')
print()

Teoretski rezultat o stacionarnoj distribuciji  broja poslova u sustavu: 
[0.6075949367088608, 0.3037974683544304, 0.0759493670886076, 0.012658227848101266]
Eksperimentalni rezultat o stacionarnoj distribuciji  broja poslova u sustavu: 
[0.6095763093071314, 0.3019147189387982, 0.07577455244033647, 0.012711796270290116]

Teoretski rezultat o očekivanom broju zauzetih procesora: 
0.4936708860759494
Eksperimentalni rezultat o očekivanom broju zauzetih procesora: 
0.4915992126303415

Teoretski rezultat o vjerojatnosti gubitka posla: 
0.012658227848101266
Eksperimentalni rezultat o vjerojatnosti gubitka posla: 
0.012712118486538346


## $ M/M/s/\infty $
Sustav s čekanjem

In [4]:
queue_network = QueueNetwork()

generator_scale = 2
consumer_scale = 4
s = 3

# ro < number_of_processors
ro = consumer_scale / generator_scale

queue = queue_network.create_queue(
    generator_function=exponential(scale=generator_scale),
    consumer_function=exponential(scale=consumer_scale),
    number_of_processors=s,
    buffer_size=np.inf
)

queue_network.set_transition(
    queue=queue,
    transition={
        -1: 1,
        0: 0
    }
)

end_time = 100000
queue_network.simulate(end_time)

pi_0 = [1 / (sum([np.power(ro, i) / math.factorial(i) for i in range(s)]) + np.power(ro, s) / (math.factorial(s - 1) * (s - ro)))]
pi_i1 = [pi_0[0] * np.power(ro, i) / math.factorial(i) for i in range(1, s)]
pi_i2 = [pi_0[0] * np.power(ro, i) / (math.factorial(s) * np.power(s, i - s)) for i in range(s, 3 * s)]
pi = pi_0 + pi_i1 + pi_i2
_pi = [queue_network.logger.calculate_stationary_distribution(0, end_time, i) for i in range(3 * s)]

print(f'Teoretski rezultat o stacionarnoj distribuciji  broja poslova u sustavu: \n{pi}')
print(f'Eksperimentalni rezultat o stacionarnoj distribuciji  broja poslova u sustavu: \n{_pi}')
print()

average_number_of_busy_processors = queue_network.logger.calculate_average_number_of_busy_processors(0, end_time)
print(f'Teoretski rezultat o očekivanom broju zauzetih procesora: \n{ro}')
print(f'Eksperimentalni rezultat o očekivanom broju zauzetih procesora: \n{average_number_of_busy_processors}')
print()

probability_of_a_job_waiting = queue_network.logger.calculate_probability_of_a_job_waiting(0, end_time, s)
print(f'Teoretski rezultat o vjerojatnosti čekanja: \n{s / (s - ro) * pi[s]}')
print(f'Eksperimentalni rezultat o vjerojatnosti čekanja: \n{probability_of_a_job_waiting}')
print()

expected_number_of_jobs_in_buffer = queue_network.logger.calculate_expected_number_of_jobs_in_buffer(0, end_time)
print(f'Teoretski rezultat o očekivanom broju poslova u spremniku: \n{ro * s / np.power(s - ro, 2) * pi[s]}')
print(f'Eksperimentalni rezultat o očekivanom broju poslova u spremniku: \n{expected_number_of_jobs_in_buffer}')
print()

expected_waiting_time_in_buffer = queue_network.logger.calculate_expected_waiting_time_in_buffer(0)
print(f'Teoretski rezultat o očekivanom trajanju čekanja u spremniku: \n{s * consumer_scale / np.power(s - ro, 2) * pi[s]}')
print(f'Eksperimentalni rezultat o očekivanom trajanju čekanja u spremniku: \n{expected_waiting_time_in_buffer}')
print()

Teoretski rezultat o stacionarnoj distribuciji  broja poslova u sustavu: 
[0.1111111111111111, 0.2222222222222222, 0.2222222222222222, 0.14814814814814814, 0.09876543209876543, 0.06584362139917695, 0.0438957475994513, 0.029263831732967534, 0.01950922115531169]
Eksperimentalni rezultat o stacionarnoj distribuciji  broja poslova u sustavu: 
[0.11810303320831572, 0.2279053401509022, 0.22407691968739343, 0.14768752625087903, 0.09772622305077086, 0.06390430232584483, 0.04283989821943547, 0.029094541767613245, 0.01699603812649677]

Teoretski rezultat o očekivanom broju zauzetih procesora: 
2.0
Eksperimentalni rezultat o očekivanom broju zauzetih procesora: 
1.9657859951038221

Teoretski rezultat o vjerojatnosti čekanja: 
0.4444444444444444
Eksperimentalni rezultat o vjerojatnosti čekanja: 
0.4299089385260381

Teoretski rezultat o očekivanom broju poslova u spremniku: 
0.8888888888888888
Eksperimentalni rezultat o očekivanom broju poslova u spremniku: 
0.8051822100364406
Teoretski rezultat o 

## $ M/M/s/m $
Sustav s čekanjem i gubitcima

In [5]:
queue_network = QueueNetwork()

generator_scale = 2
consumer_scale = 4
s = 3
m = 5

ro = consumer_scale / generator_scale

queue = queue_network.create_queue(
    generator_function=exponential(scale=generator_scale),
    consumer_function=exponential(scale=consumer_scale),
    number_of_processors=s,
    buffer_size=m
)

queue_network.set_transition(
    queue=queue,
    transition={
        -1: 1,
        0: 0
    }
)

end_time = 100000
queue_network.simulate(end_time)

pi_0 = [1 / (sum([np.power(ro, i) / math.factorial(i) for i in range(s)]) + sum([np.power(ro, i) / (math.factorial(s) * np.power(s, i - s)) for i in range(s, s + m + 1)]))]
pi_i1 = [pi_0[0] * np.power(ro, i) / math.factorial(i) for i in range(1, s)]
pi_i2 = [pi_0[0] * np.power(ro, i) / (math.factorial(s) * np.power(s, i - s)) for i in range(s, s + m + 1)]
pi = pi_0 + pi_i1 + pi_i2
_pi = [queue_network.logger.calculate_stationary_distribution(0, end_time, i) for i in range(s + m + 1)]

print(f'Teoretski rezultat o stacionarnoj distribuciji broja poslova u sustavu: \n{pi}')
print(f'Eksperimentalni rezultat o stacionarnoj distribuciji broja poslova u sustavu: \n{_pi}')
print()

average_number_of_busy_processors = queue_network.logger.calculate_average_number_of_busy_processors(0, end_time)
print(f'Teoretski rezultat o očekivanom broju zauzetih procesora: \n{ro * (1 - pi[-1])}')
print(f'Eksperimentalni rezultat o očekivanom broju zauzetih procesora: \n{average_number_of_busy_processors}')
print()

probability_of_loosing_a_job = queue_network.logger.calculate_probability_of_loosing_a_job(0)
print(f'Teoretski rezultat o vjerojatnosti gubitka posla: \n{pi[-1]}')
print(f'Eksperimentalni rezultat o vjerojatnosti gubitka posla: \n{probability_of_loosing_a_job}')
print()

Teoretski rezultat o stacionarnoj distribuciji broja poslova u sustavu: 
[0.11562252180808881, 0.23124504361617762, 0.23124504361617762, 0.15416336241078507, 0.10277557494052339, 0.06851704996034892, 0.045678033306899284, 0.030452022204599524, 0.02030134813639968]
Eksperimentalni rezultat o stacionarnoj distribuciji broja poslova u sustavu: 
[0.11416361980322218, 0.23141991463277817, 0.2331593315318793, 0.1535564047259591, 0.1027664881559158, 0.06870195325674047, 0.045501276267841874, 0.030515263963147556, 0.0202084617711283]

Teoretski rezultat o očekivanom broju zauzetih procesora: 
1.9593973037272006
Eksperimentalni rezultat o očekivanom broju zauzetih procesora: 
1.9614881221187321

Teoretski rezultat o vjerojatnosti gubitka posla: 
0.02030134813639968
Eksperimentalni rezultat o vjerojatnosti gubitka posla: 
0.020150831969993215


## Mreže redova

In [6]:
# todo